In [9]:
from open3d import *
import copy
import numpy as np
import re
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
from scipy.sparse import csr_matrix

In [14]:
scene_path = 'scene_6.ply'
model_path = 'data/Model3D/Model_pringles.ply'

In [15]:
print("Load a ply point cloud, print it, and render it")
pcd = read_point_cloud(scene_path)
# Flip it, otherwise the pointcloud will be upside down
pcd.transform([[1, 0, 0, 0], [0, -1, 0, 0], [0, 0, -1, 0], [0, 0, 0, 1]])
draw_geometries([pcd])

Load a ply point cloud, print it, and render it


In [17]:
print("Load a ply point cloud, print it, and render it")
pcd = read_point_cloud(model_path)
pcd.paint_uniform_color([1, 0.706, 0])
# Flip it, otherwise the pointcloud will be upside down
pcd.transform([[1, 0, 0, 0], [0, -1, 0, 0], [0, 0, -1, 0], [0, 0, 0, 1]])
draw_geometries([pcd])

Load a ply point cloud, print it, and render it


In [18]:
def draw_registration_result(source, target, transformation):
    source_temp = copy.deepcopy(source)
    target_temp = copy.deepcopy(target)
    #source_temp.paint_uniform_color([1, 0.706, 0])
    #target_temp.paint_uniform_color([0, 0.651, 0.929])
    source_temp.transform(transformation)
    draw_geometries([source_temp, target_temp])

In [6]:
source = read_point_cloud(model_path)
target = read_point_cloud(scene_path)
threshold = 1
trans_init = np.asarray(
            [[1, 0.011, -0.507,  0],
            [-0.139, 1, -0.215,  -1],
            [0.487, 0.255,  1, 0.8],
            [0.0, 0.0, 0.0, 1.0]])
draw_registration_result(source, target, trans_init)
print("Initial alignment")
evaluation = evaluate_registration(source, target,
        threshold, trans_init)
print(evaluation)

print("Apply point-to-point ICP")
reg_p2p = registration_icp(source, target, threshold, trans_init,
        TransformationEstimationPointToPoint())
print(reg_p2p)
print("Transformation is:")
print(reg_p2p.transformation)
print("")
draw_registration_result(source, target, reg_p2p.transformation)

Initial alignment
RegistrationResult with fitness = 1.000000, inlier_rmse = 0.267580, and correspondence_set size of 11017
Access transformation to get result.
Apply point-to-point ICP
RegistrationResult with fitness = 1.000000, inlier_rmse = 0.034289, and correspondence_set size of 11017
Access transformation to get result.
Transformation is:
[[ 0.36816138  0.72358105 -0.69449533  0.08401925]
 [-0.45977583  0.73259596  0.6753407  -0.79065387]
 [ 0.95370508  0.06985421  0.60404071  0.80911247]
 [ 0.          0.          0.          1.        ]]



In [19]:
source = read_point_cloud(model_path)
source.paint_uniform_color([1, 0.706, 0])
target = read_point_cloud(scene_path)
threshold = 1.0
trans_init = np.asarray(
            [[1, 0.011, -0.507,  0],
            [-0.139, 1, -0.215,  -1],
            [0.487, 0.255,  1, 0.8],
            [0.0, 0.0, 0.0, 1.0]])
draw_registration_result(source, target, trans_init)
print("Initial alignment")
evaluation = evaluate_registration(source, target,
        threshold, trans_init)
print(evaluation)

i=0
current_pose = trans_init
while i<10 :
    print("Apply point-to-point ICP")
    reg_p2p = registration_icp(source, target, threshold, current_pose,
            TransformationEstimationPointToPoint())
    #print(reg_p2p.fitness)
    #print("Transformation is:")
    #print(reg_p2p.transformation)
    #print("")
    draw_registration_result(source, target, reg_p2p.transformation)
    threshold = threshold/2
    current_pose = reg_p2p.transformation
    i+=1
draw_registration_result(source, target, reg_p2p.transformation)

Initial alignment
RegistrationResult with fitness = 1.000000, inlier_rmse = 0.267580, and correspondence_set size of 11017
Access transformation to get result.
Apply point-to-point ICP
Apply point-to-point ICP
Apply point-to-point ICP
Apply point-to-point ICP
Apply point-to-point ICP
Apply point-to-point ICP
Apply point-to-point ICP
Apply point-to-point ICP
Apply point-to-point ICP
Apply point-to-point ICP


In [ ]:
trans_init_pringles = np.asarray(
            [[1, 0.011, -0.507,  0],
            [-0.139, 1, -0.215,  -1],
            [0.487, 0.255,  1, 0.8],
            [0.0, 0.0, 0.0, 1.0]])

trans_init_poubelle = np.asarray(
            [[1, 0.011, -0.507,  -0.2],
            [-0.139, 1, -0.215,  0],
            [0.487, 0.255,  1, 0],
            [0.0, 0.0, 0.0, 1.0]])

In [73]:
def draw_registration_result(source, target, transformation):
    source_temp = copy.deepcopy(source)
    target_temp = copy.deepcopy(target)
    source_temp.transform(transformation)
    draw_geometries([source_temp, target_temp])

def preprocess_point_cloud(pcd, voxel_size):
    print(":: Downsample with a voxel size %.3f." % voxel_size)
    pcd_down = voxel_down_sample(pcd, voxel_size)

    radius_normal = voxel_size * 2
    print(":: Estimate normal with search radius %.3f." % radius_normal)
    estimate_normals(pcd_down, KDTreeSearchParamHybrid(
            radius = radius_normal, max_nn = 30))

    radius_feature = voxel_size * 5
    print(":: Compute FPFH feature with search radius %.3f." % radius_feature)
    pcd_fpfh = compute_fpfh_feature(pcd_down,
            KDTreeSearchParamHybrid(radius = radius_feature, max_nn = 100))
    return pcd_down, pcd_fpfh

def prepare_dataset(voxel_size):
    print(":: Load two point clouds and disturb initial pose.")
    source = read_point_cloud(model_path)
    target = read_point_cloud(scene_path)
    trans_init = np.asarray(
                [[1, 0, 0,  0],
                [0, 1, 0,  -0.5],
                [0, 0,  1, 0.8],
                [0.0, 0.0, 0.0, 1.0]])
    source.transform(trans_init)
    draw_registration_result(source, target, trans_init)

    source_down, source_fpfh = preprocess_point_cloud(source, voxel_size)
    target_down, target_fpfh = preprocess_point_cloud(target, voxel_size)
    return source, target, source_down, target_down, source_fpfh, target_fpfh

def execute_global_registration(
        source_down, target_down, source_fpfh, target_fpfh, voxel_size):
    distance_threshold = voxel_size * 1.5
    print(":: RANSAC registration on downsampled point clouds.")
    print("   Since the downsampling voxel size is %.3f," % voxel_size)
    print("   we use a liberal distance threshold %.3f." % distance_threshold)
    result = registration_ransac_based_on_feature_matching(
            source_down, target_down, source_fpfh, target_fpfh,
            distance_threshold,
            TransformationEstimationPointToPoint(False), 4,
            [CorrespondenceCheckerBasedOnEdgeLength(0.9),
            CorrespondenceCheckerBasedOnDistance(distance_threshold)],
            RANSACConvergenceCriteria(4000000, 500))
    return result

def refine_registration(source, target, source_fpfh, target_fpfh, voxel_size):
    distance_threshold = voxel_size * 0.4
    print(":: Point-to-plane ICP registration is applied on original point")
    print("   clouds to refine the alignment. This time we use a strict")
    print("   distance threshold %.3f." % distance_threshold)
    result = registration_icp(source, target, distance_threshold,
            result_ransac.transformation,
            TransformationEstimationPointToPlane())
    return result


In [74]:
voxel_size = 0.005 # means 5cm for the dataset
source, target, source_down, target_down, source_fpfh, target_fpfh = \
        prepare_dataset(voxel_size)

result_ransac = execute_global_registration(source_down, target_down,
        source_fpfh, target_fpfh, voxel_size)
print(result_ransac)
draw_registration_result(source_down, target_down,
        result_ransac.transformation)

result_icp = refine_registration(source, target,
        source_fpfh, target_fpfh, voxel_size)
print(result_icp)
draw_registration_result(source, target, result_icp.transformation)

:: Load two point clouds and disturb initial pose.
:: Downsample with a voxel size 0.005.
:: Estimate normal with search radius 0.010.
:: Compute FPFH feature with search radius 0.025.
:: Downsample with a voxel size 0.005.
:: Estimate normal with search radius 0.010.
:: Compute FPFH feature with search radius 0.025.
:: RANSAC registration on downsampled point clouds.
   Since the downsampling voxel size is 0.005,
   we use a liberal distance threshold 0.007.
RegistrationResult with fitness = 0.000000, inlier_rmse = 0.000000, and correspondence_set size of 0
Access transformation to get result.
:: Point-to-plane ICP registration is applied on original point
   clouds to refine the alignment. This time we use a strict
   distance threshold 0.002.
RegistrationResult with fitness = 0.000000, inlier_rmse = 0.000000, and correspondence_set size of 0
Access transformation to get result.
